In [ ]:
!pip install sklearn

In [ ]:
!pip install --upgrade google-cloud-storage

In [ ]:
!pip install pandas-tfrecords

In [14]:
import sqlite3
# Create a SQL connection to our SQLite database
con = sqlite3.connect(r"C:\Users\skumar\tfx_custom_component_pipeline\tfx\metadata.sqlite")
cur = con.cursor()

# The result of a "cursor.execute" can be iterated over by row
cur.execute("delete FROM Artifact where name='sklearnModel_pipe'")
con.commit()
    
cur.close()

In [3]:
import tfx 
import tensorflow as tf
import numpy as np
import pandas as pd
import pickle
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
import os
import absl

import warnings
warnings. filterwarnings('ignore')

In [4]:
_pipeline_name = 'sklearn_tfx_pipeline_xgb'
_root = r'C:\Users\skumar\tfx_custom_component_pipeline'
_tfx_root = os.path.join(_root, 'tfx')
_pipeline_root = os.path.join(_tfx_root, 'pipelines')

context = InteractiveContext(pipeline_root=_tfx_root)

In [5]:
# Example usage in a pipeline graph definition:
# ...

_data_root = os.path.join(_root, 'data')
    
example_gen = tfx.components.CsvExampleGen(input_base=_data_root)
    
context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 88
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}))

In [6]:
statistics_gen = tfx.components.StatisticsGen(
      examples=example_gen.outputs['examples'])

context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 89
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}))

In [7]:

import pandas as pd
import pandas_tfrecords as pdtfr
import os
import tensorflow as tf
from tfx import v1 as tfx
from tfx.dsl.component.experimental.decorators import component
from joblib import dump
import sklearn
from sklearn.base import BaseEstimator,TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from joblib import dump



class MultiplyColumns(BaseEstimator, TransformerMixin):
    def __init__(self, by=1, columns=None):
        self.by = by
        self.columns = columns
    
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        X= X * self.by
        return X





@component
def MyTransformerComponent(
    input_data: tfx.dsl.components.InputArtifact[tfx.types.standard_artifacts.Examples],
    sklearnPipe: tfx.dsl.components.OutputArtifact[tfx.types.standard_artifacts.Model]
    ) -> None:
    
    print(input_data.uri)
    
    # Get the URI of the output artifact representing the transformed examples, which is a directory
    example_uri = os.path.join(input_data.uri, 'Split-train')

# Get the list of files in this directory (all compressed TFRecord files)
    tfrecord_filenames = [os.path.join(example_uri, name)
                      for name in os.listdir(example_uri)]
    
    print(tfrecord_filenames)
    
    #dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")
    
    for tfrecord in tfrecord_filenames:
        data = pdtfr.tfrecords_to_pandas(tfrecord)
    
    NUMERIC_FEATURE_KEYS = [
    'age', 'capital-gain', 'capital-loss',
    'education-num', 'fnlwgt', 'hours-per-week']
    
    
    pipe = Pipeline([('scaler', StandardScaler()),
    ("multiply_cols_by_3", MultiplyColumns(3))
    ])
       
    pre_process = pipe.fit(data[NUMERIC_FEATURE_KEYS])
    tranformed_data = pre_process.transform(data[NUMERIC_FEATURE_KEYS])
    #print(model.uri)
    
    pipe_path = os.path.join(sklearnPipe.uri, 'sklearn_pipe.pkl')
    dump(pipe,pipe_path) 

transformer = MyTransformerComponent(
    input_data=example_gen.outputs['examples'])

context.run(transformer)


C:\Users\skumar\tfx_custom_component_pipeline\tfx\CsvExampleGen\examples\88
['C:\\Users\\skumar\\tfx_custom_component_pipeline\\tfx\\CsvExampleGen\\examples\\88\\Split-train\\data_tfrecord-00000-of-00001.gz']


ExecutionResult(
    component_id: MyTransformerComponent
    execution_id: 90
    outputs:
        sklearnPipe: OutputChannel(artifact_type=Model, producer_component_id=MyTransformerComponent, output_key=sklearnPipe, additional_properties={}, additional_custom_properties={}))

In [ ]:
## Trainer Component to train the model
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier

from joblib import load

@component
def MyTrainerComponent(
    input_data: tfx.dsl.components.InputArtifact[tfx.types.standard_artifacts.Examples],
    input_sklearnPipe: tfx.dsl.components.InputArtifact[tfx.types.standard_artifacts.Model],
    sklearnModel_pipe: tfx.dsl.components.OutputArtifact[tfx.types.standard_artifacts.Model]
    ) -> None:
    
    
    # Get the URI of the output artifact representing the transformed examples, which is a directory
    example_uri = os.path.join(input_data.uri, 'Split-train')

# Get the list of files in this directory (all compressed TFRecord files)
    tfrecord_filenames = [os.path.join(example_uri, name)
                      for name in os.listdir(example_uri)]
    
    print(tfrecord_filenames)
    
    #dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")
    
    for tfrecord in tfrecord_filenames:
        data = pdtfr.tfrecords_to_pandas(tfrecord)
    
    print(data.columns)
    print(data.loc[0:2,['label']])
    
    NUMERIC_FEATURE_KEYS = [
    'age', 'capital-gain', 'capital-loss',
    'education-num', 'fnlwgt', 'hours-per-week']
    
    
    _LABEL_ = 'label'
    
    
    ## Load the sklearn pipeline
    model_path = os.path.join(input_sklearnPipe.uri, 'sklearn_pipe.pkl')
    pipe = load(model_path) 
    ## Append model training step to the pipeline
    x_train = data[NUMERIC_FEATURE_KEYS]
        
    y_train = data.label
    y_train=y_train.astype('int')
    
    y_train = y_train.to_numpy()
    
    print(y_train.shape)
    print(x_train.shape)
    
    print(y_train)
    
    ## Train random clasifier model
    estimator = XGBClassifier(
                            verbose=True)
    
    pipe.steps.append(['Classifier',estimator])
    
    pipe.fit(x_train, y_train)
    
    model_path = os.path.join(sklearnModel_pipe.uri, 'model_pipeline.pkl')
    dump(estimator,model_path) 
    
    
    
trainer = MyTrainerComponent(
    input_data=example_gen.outputs['examples'],
    input_sklearnPipe=transformer.outputs['sklearnPipe']
)

context.run(trainer)

['C:\\Users\\skumar\\tfx_custom_component_pipeline\\tfx\\CsvExampleGen\\examples\\88\\Split-train\\data_tfrecord-00000-of-00001.gz']


# Create Pipeline


In [ ]:
from tfx.orchestration import pipeline
from tfx.proto.orchestration import pipeline_pb2
from tfx.orchestration.local.local_dag_runner import LocalDagRunner

metadata_path=r"C:\Users\skumar\tfx_custom_component_pipeline\tfx\metadata.sqlite"



def create_pipeline(
    pipeline_name,
    pipeline_root,
    data_root
     ) -> pipeline.Pipeline:
    
    #1st Component  (Load data)
    example_gen = tfx.components.CsvExampleGen(input_base=_data_root)
    
    #2nd Component (Generate stats)
    statistics_gen = tfx.components.StatisticsGen(
      examples=example_gen.outputs['examples'])

    #3rd Componenet (Create sklearn data processing pipeline)
    transformer = MyTransformerComponent(
    input_data=example_gen.outputs['examples'])
    

    #4th Componenet (create sklearn model)
    trainer = MyTrainerComponent(
    input_data=example_gen.outputs['examples'],
    input_model=transformer.outputs['sklearnPipe'])

    
    #5th compoenent (push to serving directory)



    return pipeline.Pipeline(
      pipeline_name=pipeline_name,
      pipeline_root=pipeline_root,
      components=[
          example_gen,
          statistics_gen,
          transformer,
          trainer
      ],
      enable_cache=True,
      metadata_connection_config=tfx.orchestration.metadata.
      sqlite_metadata_connection_config(metadata_path),
  )


# To run this pipeline from the python CLI:
#   $python penguin_pipeline_sklearn_local.py

#absl.logging.set_verbosity(absl.logging.INFO)
LocalDagRunner().run(
  create_pipeline(pipeline_name=_pipeline_name,
                   pipeline_root=_pipeline_root,
                   data_root=_data_root))
